## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,2021-11-01 04:39:00,71.9167,114.0833,2.32,97,97,5.21,overcast clouds
1,1,Tuktoyaktuk,CA,2021-11-01 04:39:01,69.4541,-133.0374,30.20,80,75,14.97,broken clouds
2,2,Hermanus,ZA,2021-11-01 04:39:01,-34.4187,19.2345,61.81,74,93,13.80,light rain
3,3,Atuona,PF,2021-11-01 04:35:20,-9.8000,-139.0333,78.84,73,0,18.92,clear sky
4,4,Punta Arenas,CL,2021-11-01 04:39:02,-53.1500,-70.9167,46.51,57,0,8.05,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? \n"))
max_temp = float(input("What is the maximum temperature you would like for your trip? \n"))

What is the minimum temperature you would like for your trip? 
75
What is the maximum temperature you would like for your trip? 
90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Atuona,PF,2021-11-01 04:35:20,-9.8000,-139.0333,78.84,73,0,18.92,clear sky
6,6,Ahuimanu,US,2021-11-01 04:39:04,21.4447,-157.8378,80.31,71,90,4.61,overcast clouds
15,15,Butaritari,KI,2021-11-01 04:39:09,3.0707,172.7902,82.58,73,11,12.12,light rain
20,20,Vaini,TO,2021-11-01 04:39:11,-21.2000,-175.2000,80.76,83,20,9.22,few clouds
22,22,Kapaa,US,2021-11-01 04:39:12,22.0752,-159.3190,78.78,82,75,0.00,broken clouds
27,27,Filingue,NE,2021-11-01 04:39:14,14.3521,3.3168,81.75,17,100,6.87,overcast clouds
28,28,Mahebourg,MU,2021-11-01 04:39:15,-20.4081,57.7000,79.25,78,40,6.91,scattered clouds
33,33,Gigmoto,PH,2021-11-01 04:39:17,13.7796,124.3900,84.47,66,38,12.73,scattered clouds
42,42,Cayenne,GF,2021-11-01 04:39:21,4.9333,-52.3333,82.09,83,75,3.00,broken clouds
43,43,Olinda,BR,2021-11-01 04:39:21,-8.0089,-34.8553,80.33,74,40,6.91,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Country                206
Date                   206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID                206
City                   206
Country                206
Date                   206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Atuona,PF,78.84,clear sky,-9.8000,-139.0333,
6,Ahuimanu,US,80.31,overcast clouds,21.4447,-157.8378,
15,Butaritari,KI,82.58,light rain,3.0707,172.7902,
20,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
22,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
27,Filingue,NE,81.75,overcast clouds,14.3521,3.3168,
28,Mahebourg,MU,79.25,scattered clouds,-20.4081,57.7000,
33,Gigmoto,PH,84.47,scattered clouds,13.7796,124.3900,
42,Cayenne,GF,82.09,broken clouds,4.9333,-52.3333,
43,Olinda,BR,80.33,scattered clouds,-8.0089,-34.8553,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             206
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))